In [5]:
import numpy as np
import random
import time
import matplotlib.pyplot as plt
from qiskit import *
from qiskit.optimization.applications.ising import tsp
from qiskit.optimization.converters import IsingToQuadraticProgram
from qiskit.optimization.algorithms import MinimumEigenOptimizer,GroverOptimizer
from qiskit.aqua.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.aqua.components.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
from qiskit.aqua import QuantumInstance
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.aqua.algorithms import QAOA
from qiskit.optimization import QuadraticProgram
from itertools import permutations

In [10]:
def brute_force_tsp(w, N):
    a=list(permutations(range(1,N)))
    last_best_distance = 1e10
    for i in a:
        distance = 0
        pre_j = 0
        for j in i:
            distance = distance + w[j,pre_j]
            pre_j = j
        distance = distance + w[pre_j,0]
        order = (0,) + i
        if distance < last_best_distance:
            best_order = order
            last_best_distance = distance
    return last_best_distance, best_order
ins = tsp.random_tsp(5, seed=123)
brute_force_tsp(ins.w, ins.dim)

(271.0, (0, 1, 2, 4, 3))

In [12]:
def get_qubo(ins):
    qubitOp, offset = tsp.get_operator(ins)
    qubo = QuadraticProgram()
    qubo.from_ising(qubitOp, offset, linear=True)
    return qubitOp, offset, qubo

In [16]:
def result_exact(qubo):
    exact_mes = NumPyMinimumEigensolver()
    exact = MinimumEigenOptimizer(exact_mes)
    exact_result = exact.solve(qubo)
    return exact_result
ins = tsp.random_tsp(4, seed=123)
qubitOp, offset, qubo = get_qubo(ins)
result_exact(qubo)

optimal function value: 236.0
optimal value: [0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0.]
status: SUCCESS